In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import glob

# 1. Define CNN Architecture
model = Sequential()

# Convolutional Layer 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3))) 
model.add(MaxPooling2D((2, 2)))

# Convolutional Layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Convolutional Layer 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid')) 

# 2. Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary() 

# 3. Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Assuming you have your data organized into training and validation folders
train_image_paths = glob.glob('path/to/your/train/data/**/*.jpg', recursive=True)  # Adjust the pattern
validation_image_paths = glob.glob('path/to/your/validation/data/**/*.jpg', recursive=True) # Adjust the pattern

train_set = train_datagen.flow_from_directory(
    'C:/Users/Lenovo/Desktop/PetImages',  # Replace with your actual path
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_set = validation_datagen.flow_from_directory(
    'C:/Users/Lenovo/Desktop/PetImages',  # Replace with your actual path
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# 4. Define Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras',  # Changed to '.keras' extension
                                    save_best_only=True, 
                                    monitor='val_accuracy', 
                                    mode='max')

# 5. Train the Model
history = model.fit(
    train_set,
    epochs=20,  # Adjust based on your training needs
    validation_data=validation_set,
    callbacks=[early_stopping, model_checkpoint]
)

# 6. Evaluate the Model
loss, accuracy = model.evaluate(validation_set)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# 7. Plot Training History
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()